In [0]:
#importing libraries

import numpy as np #for mathematical calculations if any needed 
import pandas as pd #for playing with dataframe
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer #For tokenizing the words
from keras.preprocessing.sequence import pad_sequences #for padding the words of same length
from keras.models import Sequential #for building a sequence models
from keras.layers import Embedding, Bidirectional, LSTM, Dense #for defining a layers inside the model
from sklearn.model_selection import train_test_split
import csv
import re #for manipulating with regex
import nltk #importing stopwords to be removed from the dataset
from gensim.models import Word2Vec #for creating word embeddings using Word2Vec model(CBAG or Skipgram)/ Loading pretrained Word2Vec or Glove Model.

Using TensorFlow backend.


In [0]:
#importing the dataset
train_df = pd.read_csv("/content/drive/My Drive/Movie_Reviews/RNN/train_cleaned.csv")
test_df = pd.read_csv("/content/drive/My Drive/Movie_Reviews/RNN/test_cleaned.csv")

In [0]:
#input declearations 
embedding_size = 50
sequence_length = 100
corpus_size = 5000

In [0]:
#setting up stopwords from the nltk library
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
#Cleaning the Train Dataset
#train_df['reviews'] = train_df['reviews'].apply(lambda x: re.sub('<br />',' ',x)) #removing nextline. If it is in another format, please check and remove it.
train_df['reviews'] = train_df['reviews'].apply(lambda x: re.sub('<br />',' ',x)) #removing nextline. If it is in another format, please check and remove it.
train_df['reviews'] = train_df['reviews'].apply(lambda x: re.sub('[^a-zA-z\s]','',x)) #removing all charecters except alphabets
train_df['reviews'] = train_df['reviews'].apply(lambda x: re.sub(r'\s+',' ',x)) #removing extra spaces
train_df['reviews'] = train_df['reviews'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)])) #removing stopwords

In [0]:
#Cleaning the test Dataset
#test_df['reviews'] = test_df['reviews'].apply(lambda x: re.sub('<br />',' ',x)) #removing nextline. If it is in another format, please check and remove it.
test_df['reviews'] = test_df['reviews'].apply(lambda x: re.sub('<br />',' ',x)) #removing nextline. If it is in another format, please check and remove it.
test_df['reviews'] = test_df['reviews'].apply(lambda x: re.sub('[^a-zA-z\s]','',x)) #removing all charecters except alphabets
test_df['reviews'] = test_df['reviews'].apply(lambda x: re.sub(r'\s+',' ',x)) #removing extra spaces
test_df['reviews'] = test_df['reviews'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)])) #removing stopwords

In [0]:

#Creating_Embedding_matix
sentences = train_df['reviews'].tolist() #converting reviews in dataframe to list.
sentences = [line.lower().split(' ') for line in sentences] #Again convert each sentence in a list to list of lists. #Also each word to lowercase 

In [0]:

"""
sentences - It should be list of lists
min_count - minimum number of times a word should be present to create wordvectors
window - number of words taken to train on either side of the word.
size - embedding layer size
workers - Number of cores it can parallely work on.
sg - #1 for skipgram or #0 for Contionous Bag of words Model
"""
embed_model = Word2Vec(sentences, window=5, min_count=1, size=embedding_size, workers=2, sg=0) 

In [0]:
#Training Embedding Matrix
embed_model.train(sentences,total_examples=len(sentences),epochs=10)

(28591087, 30332070)

In [0]:
"""# save model
embed_model.save('drive/My Drive/Movie_Reviews/RNN/skip_gram_model.bin')
# load model
skipgram_embed_model = Word2Vec.load('drive/My Drive/Movie_Reviews/RNN/skip_gram_model.bin')"""

embed_model.save('drive/My Drive/Movie_Reviews/RNN/CBAG_model.bin')
# load model
embed_model = Word2Vec.load('drive/My Drive/Movie_Reviews/RNN/CBAG_model.bin')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
print(embed_model)

Word2Vec(vocab=108991, size=50, alpha=0.025)


In [0]:
"""The num_words in Tokenizer, the maximum number of words to keep, based on word frequency. 
If num_words = 10000 Only the most common 9999 words will be kept. all he extra words will be removed."""
tokenizer = Tokenizer(num_words = corpus_size) #setting up tokenizer
b=tokenizer.fit_on_texts(train_df['reviews']) #fitting tokenizer on dataframe 
X_train = tokenizer.texts_to_sequences(train_df['reviews']) # removing least repeated words and converting them into sequence of numbers.

In [0]:
print('Maximum review length: {}'.format(len(max(X_train, key=len)))) #checking maximum review length
print('Minimum review length: {}'.format(len(min(X_train, key=len)))) #checking minimum review length

Maximum review length: 933
Minimum review length: 3


In [0]:
"""Pads sequences to the same length.
This function transforms a list of num_samples sequences (lists of integers) into a 2D Numpy array of 
shape (num_samples, num_timesteps). num_timesteps is either the maxlen argument if provided, or the 
length of the longest sequence otherwise. Sequences that are shorter than num_timesteps are padded with value at the end.
Sequences longer than num_timesteps are truncated so that they fit the desired length. 
The position where padding or truncation happens is determined by the arguments padding and truncating, respectively.
Pre-padding is the default."""

X_train = pad_sequences(X_train,maxlen = sequence_length)
print(X_train.shape)

(25000, 100)


In [0]:
embedding_matrix = np.zeros((corpus_size, embedding_size))


In [0]:
for word, i in tokenizer.word_index.items():
	embedding_vector = embed_model.wv[word]
	if embedding_vector is not None and i <=(corpus_size-1):
		embedding_matrix[i] = embedding_vector

In [0]:
word="idnt"
if word in list(embedding_matrix_50_loaded.index):
  print (word)

NameError: ignored

In [0]:
#loading Glove model
#File to Required Embedding Matrix
embedding_matrix_50_loaded = pd.read_table("drive/My Drive/EmbeddingDownloaded/glove.6B.50d.txt", sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)
for word, i in tokenizer.word_index.items():
  if i <=(corpus_size-1):
    if word in list(embedding_matrix_50_loaded.index):
      embedding_vector = embedding_matrix_50_loaded.loc[word]
      embedding_matrix[i] = np.asarray(embedding_vector)
    else:
      print (word)


itbr
moviebr
filmbr
timebr
clichs
lowbudget
thembr
theyve
himbr
thisbr
allbr
storybr
onebr
thatbr
mebr
wellbr
yearold
overthetop
wouldve
outbr
waybr
herebr
isbr
bmovie
clichd
lifebr
couldve
endbr
writerdirector
onbr
againbr
goodbr
therebr
youbr
badbr
oneliners
toobr
herbr
mstk
moviesbr
theyd
betterbr
reallife
mustsee
theyll
worldbr
filmsbr
funnybr
upbr
charactersbr
spoilersbr
timesbr
workbr
showbr
watchbr
itll
morebr
notbr
everyones
scenebr
seenbr
etcbr
screenbr
wasbr
scenesbr
madefortv
onedimensional
inbr
bebr
peoplebr
muchbr
seriesbr
plotbr
eitherbr
offbr
aboutbr
minutesbr
funbr
withbr
middleaged
daybr
otherbr
wrongbr
actingbr
yearsbr
nowbr
audiencebr
characterbr
thingbr
madebr
wellwritten
thoughtprovoking
togetherbr


In [0]:
embedding_matrix.shape

(5000, 50)

In [0]:
Y_train = train_df['pos_or_neg']
Y_test = test_df['pos_or_neg']

In [0]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train,Y_train, random_state=0, test_size=0.2)
print(X_train.shape, X_valid.shape, Y_train.shape, Y_valid.shape )

(20000, 100) (5000, 100) (20000,) (5000,)


In [0]:
print(Y_train.shape)
print(Y_test.shape)

(20000,)
(25000,)


In [0]:
X_test = tokenizer.texts_to_sequences(test_df['reviews']) #tokenizing the test data
X_test = pad_sequences(X_test,maxlen = sequence_length)

In [0]:
#Building the model
model=Sequential()
#model.add(Embedding(input_dim=corpus_size,output_dim=embedding_size""",weights=[embedding_matrix],""",input_length=sequence_length,trainable=True))
model.add(Embedding(input_dim=corpus_size,output_dim=embedding_size,input_length=sequence_length,trainable=True))
model.add(Bidirectional(LSTM(units=128)))
model.add(Dense(1,activation="sigmoid"))                           

In [0]:
model.compile(loss="binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 50)           250000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               183296    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 433,553
Trainable params: 433,553
Non-trainable params: 0
_________________________________________________________________


In [0]:
#Declaring filepath for saving weigths
filepath = "weigths.best.hdf5"

In [0]:
#Declaring filepath for saving weigths
filepath = "weigths.best.hdf5"

#creating a checkpoint
checkpoint = ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only = True, mode = 'max')

In [0]:
model.fit(X_train,Y_train, validation_data = [X_valid,Y_valid], epochs = 15, batch_size = 2500, callbacks=[checkpoint])

Train on 20000 samples, validate on 5000 samples
Epoch 1/15
20000/20000 [==============================] - 8s 389us/step - loss: 0.5877 - acc: 0.7548 - val_loss: 0.5701 - val_acc: 0.7618

Epoch 00001: val_acc improved from -inf to 0.76180, saving model to weigths.best.hdf5
Epoch 2/15
20000/20000 [==============================] - 8s 377us/step - loss: 0.5128 - acc: 0.7808 - val_loss: 0.4634 - val_acc: 0.7970

Epoch 00002: val_acc improved from 0.76180 to 0.79700, saving model to weigths.best.hdf5
Epoch 3/15
20000/20000 [==============================] - 8s 378us/step - loss: 0.3989 - acc: 0.8289 - val_loss: 0.3854 - val_acc: 0.8426

Epoch 00003: val_acc improved from 0.79700 to 0.84260, saving model to weigths.best.hdf5
Epoch 4/15
20000/20000 [==============================] - 8s 380us/step - loss: 0.3412 - acc: 0.8653 - val_loss: 0.3574 - val_acc: 0.8512

Epoch 00004: val_acc improved from 0.84260 to 0.85120, saving model to weigths.best.hdf5
Epoch 5/15
20000/20000 [==================

In [0]:
scores = model.evaluate(X_test,Y_test,verbose=0)
print(model.metrics_names[1], scores[1]*100)

acc 83.636


In [0]:
All 50D

#Skipgram-Word2vec - NonTrainable Parameters - Accuracy 85.124
#Skipgram-Word2vec - Trainable Parameters - Accuracy 83.52
#CBAG-Word2vec - NonTrainable Parameters - Accuracy 85.488
#CBAG-Word2vec - Trainable Parameters - Accuracy 85.916
#default kears embedding - 82.888
#Glove pretrained - Accuracy 83.248

In [0]:
os.mknod("model.json")

drive  sample_data  weigths.best.hdf5


In [0]:
#Saving the model
model_json = model.to_json()
with open("model.json",'w') as json_file:
  json_file.write(model_json)

In [0]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)
print("Loaded model from disk")

In [0]:
# load weights into new model
loaded_model.load_weights("weigths.best.hdf5")
print("Loaded weigths from disk")

In [0]:
# evaluate loaded model on test data
loaded_model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])
score = loaded_model.evaluate(X_test, Y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))